In [2]:
#import statements
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

#from keypass import NOAA_api
import pylab as pl
from IPython import display

from functools import partial
import pyproj
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score
from numpy import absolute,mean,std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive
from IPython.display import Image
from IPython.core.display import HTML 

# Modeling CO2 Sequestration using the Global Ocean Data Analysis Project (GLODAP) 

[GitHub Repo](https://github.com/ossana1/DATA606_FinalProject)<br><br>
[Project Site](https://sites.google.com/s/14-zXY-tR-4ddTR09NcwHH0VdedwqjA0Q/p/1mnouXuUqS3ud_088LqPGmpKfiSUgTiD3/edit?ths=true)

### Project Goal: Carbon Dioxide Concentration Modeling to Fill Dataset Gaps 

The GLODAP v2.2020 dataset has approximately 65% of entries missing $tCO_2$values. The goal of this project is to use a regression ML model to fill the missing entries in the dataset, using measurements of pH, alkalinity, geolocation, temperature, date, etc. as model inputs. 

Increasing the robustness of this data set could help predict future net carbon dioxide absorption areas which may be at higher risk for earlier ocean acidification than predicted. It may be possible to protect these areas in ways that slow or mitigate the changes in TCO2  that can damage marine life and a valuable food source for mankind. <br>
<img src="https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/images/ProjectMap.png" 
     height="1627.19" width="1005.12" >

***
Model performance on train, test, and validation sets for $tCO_2$ ML random forest regression model ($R^2$ and explained variance of ~0.99 on all sets). The mean square error of the train, test, validation sets were 9.84, 77.0, 68.7, respectively.

<img src="https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/images/MLperformance.png">

### A smaller version of the original dataset is available to view  in the first visual. Use this visual to see a condensed version of the data in the data set after the $tCO_2$ is filled with ML RF model.

Every 25th row is saved to make the dashboard useable (ie faster). There were 1218966 rows before the trim and 48759 after. There are 33 cruises with <100 datapoints out of 936 cruises, this data loss is acceptable to make the dashboard respond to users in real time.  

Select the dataset variable column and year using the dropdowns.

In [ ]:
#https://stackoverflow.com/questions/60609760/is-there-a-way-to-import-csv-file-from-github-automatically-to-my-jupyter-notebo (accessed Oct. 08, 2020).

#df =pd.read_csv('C:\\Users\\ossan\\DATA606\\DATA\\TCO2_filled.csv',index_col=0) #ORIGINAL
#plotly is too slow with complete set -use this for now 
url = 'https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/data/TCO2_small.csv'
df = pd.read_csv(url,index_col=0)
dftrim = df 
#print(len(df))
df['year'] =pd.to_numeric( df['year'])
df['year'] =df['year'].astype(int)
g=df.groupby('year')

In [ ]:
frames = {} # make dictionary of dataframes per year
for x,y in g:
    frames[x] = y
#frames[list(frames.keys())[-1]]

In [ ]:
#make variable list for viusla 
var = df.columns 
years = list(frames.keys())
list_updatemenus=[];temp_dict={}
#make dictionaries for dropdowns for visual
for n, year in enumerate(years):
    visible = [False] * len(years)
    visible[n] = True
    temp_dict = dict(label = str(year),method = 'update',
                 args = [{'visible': visible},{'title': 'Year %d' % year}])
    list_updatemenus.append(temp_dict)
    
list_c=[];c_dict={}
for n, year in enumerate(var):
    visible = [False] * len(var)
    visible[n] = True
    c_dict = dict(label = str(var),method = 'update',
                 args = [{'visible': visible}, {'title': var}])
    list_c.append(c_dict)
    

In [ ]:
#make list for nicer formatted columns for visuals 
col=['Cruise', 'Station',  'Year', 'Month', 'Day', 'Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth',
       'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']

actu=['cruise', 'station', 'year', 'month', 'day', 'bottomdepth', 'maxsampdepth', 'pressure', 'depth',
       'temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']
unit=[ '', 'm', 'm','bar', 'm', 'C', 'C', '', 'kg m-3',
       'kg m-3', 'kg m-3', 'kg m-3', 'kg m-3', 'umol kg-1', 'umol kg-1', 'umol kg-1','umol kg-1', 'umol kg-1', 'umol kg-1', 'umol kg-1', '', '',
       'umol kg-1']
def plot_compare_lag(var, year):
    #make figure 
    fig5 =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=dftrim["longitude"],
                                  lat=dftrim["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 2, color = dftrim[actu[col.index(var)]], colorscale = 'Inferno',
            cmax = df[actu[col.index(var)]].max(),colorbar_title=unit[col.index(var)]) ),
    layout= dict(title ='Variable: ' +var + ' | Year:'  + str(year.astype(str)[0:4]))  )
    fig5.show()
    return fig5 

interactive(plot_compare_lag, var=col, 
            year=list(np.linspace(1984,2019,(2019-1983))),  )


***
## See the changes over the years with this visual. 

The complete (filled) dataset is first grouped by latitude and longitude and averaged per year. The latitude and longitude of the ending year in the query is used with sklearn BallTree to find (haversine distance) the closest cruise geolocation in the startinng year. The difference between the starting year and end year variable of choice is displayed (end year- start year value). One shortcoming is that depth of the samples are not taken into account at this time.

In [5]:
url = 'https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/data/Groupedlatlongyr.csv'
latlong= pd.read_csv(url,index_col=0)
latlong.drop(columns=['cruise','station','day','month','Fill','cast'],inplace=True)
#clean up columns for labels 
colu=['Year','Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth', 'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']
actua=[ 'year', 'bottomdepth', 'maxsampdepth','pressure', 'depth', 'temperature', 'theta', 'salinity', 'sigma0',
       'sigma1', 'sigma2', 'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate','nitrite', 'silicate', 'phosphate', 'talk', 'phts25p0', 'phtsinsitutp',
       'tco2']

unit=[ '', 'm', 'm','bar', 'm', 'C', 'C', '', 'kg m-3',
       'kg m-3', 'kg m-3', 'kg m-3', 'kg m-3', 'umol kg-1', 'umol kg-1', 'umol kg-1','umol kg-1', 'umol kg-1', 'umol kg-1', 'umol kg-1', '', '',
       'umol kg-1']


In [7]:
%matplotlib inline

from sklearn.neighbors import BallTree

def chang(var,start,end,df):
    start_year=latlong[latlong.year ==start].reset_index(); end_year=latlong[latlong.year ==end].reset_index()
    query_lats = end_year.latitude;query_lons = end_year.longitude
    
    varx = actua[colu.index(var)] 
    #search for closest lat and long 
    ###https://stackoverflow.com/questions/10549402/kdtree-for-longitude-latitude
    bt = BallTree(np.deg2rad(start_year[['latitude', 'longitude']].values), metric='haversine')
    distances, indices = bt.query(np.deg2rad(np.c_[query_lats, query_lons]))

    i = pd.DataFrame(list(zip(distances,indices)),columns=['distances','indices'])
    end_year['compare']  =i.iloc[:,1].astype(int)
    end_year['data'] = start_year.loc[end_year.compare,varx].values
    end_year['diff'] =(end_year[varx]- end_year['data'])
    return end_year

def plot_compare_lag(var, startyear,endyear):
    change = chang(var,startyear,endyear,latlong)
    fig =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=change["longitude"],
                                  lat=change["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 4, color = change['diff'], colorscale = 'Inferno',
            colorbar_title=' Change' +' '+unit[col.index(var)] ) ), #cmax = 1.5*change['diff'].max(),
    layout= dict(title ='Variable Change: ' +var + ' | Start Year:'  + 
                 str(startyear.astype(str)[0:4]) +'| End Year:'  + 
                 str(endyear.astype(str)[0:4]))  )
    fig.show()
    return fig 



interactive(plot_compare_lag, var=colu, startyear=list(np.linspace(1984,2019,(2019-1983))), 
            endyear=list(np.linspace(1984,2019,(2019-1983))),)


interactive(plot_compare_lag, var=colu, startyear=list(np.linspace(1984,2019,(2019-1983))), 
            endyear=list(np.linspace(1984,2019,(2019-1983))),)


interactive(children=(Dropdown(description='var', options=('Year', 'Bottom Depth', 'Max Sample Depth', 'Pressu…